# 🎬 AI Video Clipper V3 - Google Colab (CPU Mode)

**Gerador Automático de Clips Virais - 100% CPU**

- ✅ **Não precisa de GPU** - Roda na CPU gratuita
- ✅ Transcrição em Português (VOSK PT-BR)
- ✅ Interface Web via Ngrok
- ✅ Salva clips no Google Drive

---

## ⚙️ ETAPA 1: Configurar Runtime CPU

**IMPORTANTE:** Vá em `Runtime → Change runtime type → None (CPU)`

Isso garante que você não gaste créditos de GPU desnecessariamente.

## 📦 ETAPA 2: Instalar Tudo Automaticamente

Execute esta célula uma vez. Ela vai:
1. Clonar o repositório
2. Instalar FFmpeg
3. Instalar bibliotecas Python
4. Baixar modelo de transcrição PT-BR (~50MB)

In [ ]:
%%time
# ========================================
# INSTALAÇÃO COMPLETA (CPU OPTIMIZED)
# ========================================

import os

# 1. Clonar repositório
print("📥 Clonando repositório...")
if not os.path.exists('AI-Video-Clipper-Studio-V3'):
    !git clone --depth 1 https://github.com/noviso123/AI-Video-Clipper-Studio-V3.git
os.chdir('AI-Video-Clipper-Studio-V3')

# 2. Instalar dependências do sistema
print("\n🔧 Instalando FFmpeg...")
!apt-get update -qq && apt-get install -y ffmpeg libsndfile1 espeak-ng -qq

# 3. Instalar Python packages (CPU only - sem CUDA)
print("\n🐍 Instalando bibliotecas Python...")
!pip install -q --no-cache-dir \
    flask==3.0.0 flask-cors==4.0.0 \
    vosk pyngrok yt-dlp \
    moviepy==1.0.3 pydub Pillow \
    requests tqdm edge-tts pyttsx3 soundfile

# 4. Baixar modelo VOSK PT-BR
print("\n🧠 Baixando modelo de transcrição PT-BR...")
!python download_models.py

# 5. Criar pastas
!mkdir -p exports temp logs

print("\n" + "="*50)
print("✅ INSTALAÇÃO CONCLUÍDA!")
print("="*50)

## 💾 ETAPA 3: Conectar Google Drive (Salvar Clips)

In [ ]:
from google.colab import drive
import os

# Montar Google Drive
print("📂 Conectando ao Google Drive...")
drive.mount('/content/drive')

# Criar pasta para clips
DRIVE_EXPORT = '/content/drive/MyDrive/AI-Video-Clipper-Exports'
os.makedirs(DRIVE_EXPORT, exist_ok=True)

# Link simbólico para salvar direto no Drive
!rm -rf exports
!ln -s {DRIVE_EXPORT} exports

print(f"\n✅ Clips serão salvos em: {DRIVE_EXPORT}")

## 🌐 ETAPA 4: Iniciar Interface Web

Execute esta célula e clique no link que aparecer para acessar a interface!

In [ ]:
from pyngrok import ngrok
import subprocess
import time

# Token Ngrok (já configurado)
NGROK_TOKEN = "2tvNFAWzP9KMYZGpfCqx1EQmmwN_NPCQKjeqHD7pomCtJFVA"
ngrok.set_auth_token(NGROK_TOKEN)

# Configurar ambiente
with open('.env', 'w') as f:
    f.write('FLASK_HOST=0.0.0.0\n')
    f.write('FLASK_PORT=5000\n')
    f.write('DISABLE_SSL_VERIFY=True\n')

# Iniciar servidor Flask
print("🚀 Iniciando servidor...")
flask = subprocess.Popen(
    ['python', 'app.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    bufsize=1,
    universal_newlines=True
)
time.sleep(6)

# Criar túnel público
try:
    ngrok.kill()
except:
    pass

url = ngrok.connect(5000)

print("\n" + "🔗"*30)
print(f"\n✅ INTERFACE ONLINE!")
print(f"\n👉 ACESSE: {url}")
print("\n" + "🔗"*30)
print("\n⚠️ MANTENHA ESTA CÉLULA RODANDO!")
print("\n📋 Logs:")

# Mostrar logs do servidor
try:
    for line in flask.stdout:
        print(line.strip())
except KeyboardInterrupt:
    flask.terminate()
    print("\n🛑 Servidor parado.")

---

## 🎥 Alternativa: Processar Vídeo por Código

Se preferir não usar a interface web:

In [ ]:
# Descomente e edite para processar:

# VIDEO_URL = "https://www.youtube.com/watch?v=XXXXXXXX"
# !python main.py --url "{VIDEO_URL}" --clips 3 --min-duration 30 --max-duration 90

print("💡 Edite a URL acima e descomente para processar")

---

## 📤 Fazer Upload de Vídeo Local

In [ ]:
from google.colab import files

print("📤 Selecione um vídeo do seu computador:")
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"\n✅ Arquivo: {filename}")
    print(f"   Tamanho: {len(uploaded[filename])/1024/1024:.1f} MB")
    
    # Mover para pasta temp
    with open(f'temp/{filename}', 'wb') as f:
        f.write(uploaded[filename])
    
    print(f"\n▶️ Para processar, use na interface: temp/{filename}")